## Preparation

In [1]:
# Set path
import pickle
path = 'C:/Users/User/Desktop/Assignatures/Synthesis project/GraphAnomaly/dades_arnau/'

In [2]:
# Load train and test set
with open(path + 'sequential_train.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open(path + 'sequential_test.pkl', 'rb') as f:
    test_set = pickle.load(f)

In [3]:
from Helper import separate_labels

# Split labels
X_train, y_train, X_test, y_test = separate_labels(train_set, test_set)

In [1]:
#import torch
import torch.nn as nn

: 

In [1]:
class DevNetworkD(nn.Module):
    def __init__(self, input_shape):
        super(DevNetworkD, self).__init__()
        #self.l2_lambda = 0.01
        self.dmodel = nn.Sequential(
            nn.Linear(input_shape, 1000),
            nn.ReLU(),
            nn.Linear(1000, 250),
            nn.ReLU(),
            nn.Linear(250, 20),
            nn.ReLU(),
            nn.Linear(20, 1)
        )
    
    def forward(self, x):
        return self.dmodel(x)
    
    # def l2_regularization(self):
    #     l2_reg = torch.tensor(0., requires_grad=True)
    #     for param in self.parameters():
    #         l2_reg += torch.norm(param, 2)
    #     return self.l2_lambda * l2_reg


: 

In [ ]:
from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    DevNetwork,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X_train, y_train)